## Full PF segmentation - superblock assembly

Assemble segmentation superblocks from even-sized blocks that have already been stitched and share IDs

In [1]:
import json
import time
import math
from cerebellum.data_prep.seg_prep import *

with open('data_locs.json') as f:
	data_locs = json.load(f)
    
linked_block_file = data_locs["initial-seg"]["dir"]+data_locs["initial-seg"]["8nm-pf-linked"] # first block file
block_shape = h5py.File(linked_block_file,'r')['main'].shape
print "Original block size:", block_shape
resolution = (30,48,48) # order Z, Y, X
dsmpl = (1,6,6)
dsmpl_block_shape = tuple([int(math.ceil(float(block_shape[i])/dsmpl[i])) for i in range(3)])
print "Downsampling by", dsmpl, "to:", dsmpl_block_shape

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Original block size: (90, 3248, 6960)
Downsampling by (1, 6, 6) to: (90, 542, 1160)


In [2]:
# generate superblocks for full lateral extent
save_folder = './data/superblocks_haidong/'
save_fname = 'pred_full_superblock'
n_blocks = 26 # total number of blocks to assemble
superblock_steps = [2, 4, 8, 12, 26] # save after assembling these many blocks
assert max(superblock_steps)==n_blocks
superblock = np.zeros((n_blocks*dsmpl_block_shape[0], 
                       dsmpl_block_shape[1], dsmpl_block_shape[2]), dtype=np.uint32) # allocate max size
total_runtime = 0
for i in range(n_blocks):
    block_time = time.time()
    zz = i*data_locs["block-size"]+data_locs["aff-offset"]
    if zz!=data_locs["aff-offset"]: # adjust block index
        linked_block_file = linked_block_file[:-7]+"%04d.h5"%(zz)
    print "Appending the %dth block: "%(i) + linked_block_file
    seg_block = SegPrep("temp", resolution)
    seg_block.read(linked_block_file, "main", dsmpl=dsmpl)
    print "Finished reading block"
    superblock[i*block_shape[0]:(i+1)*block_shape[0],:,:] = seg_block.data
    block_runtime = time.time()-block_time
    print "Finished operations with block %d in %f seconds"%(i, block_runtime)
    total_runtime += block_runtime
    print "Total runtime so far: %f seconds"%(total_runtime)
    if i+1 in superblock_steps:
        print "Saving superblock with %d blocks"%(i+1)
        writeh5(save_folder+save_fname+ '_%dblocks.h5'%(i+1), 'main', 
                superblock[:(i+1)*block_shape[0],:,:], compression='gzip')

Appending the 0th block: /mnt/hp03/donglai/public/cere_pf/data_link_strict/0014.h5
Finished reading block
Finished operations with block 0 in 107.343472 seconds
Total runtime so far: 107.343472 seconds
Appending the 1th block: /mnt/hp03/donglai/public/cere_pf/data_link_strict/0104.h5
Finished reading block
Finished operations with block 1 in 105.237848 seconds
Total runtime so far: 212.581320 seconds
Saving superblock with 2 blocks
Appending the 2th block: /mnt/hp03/donglai/public/cere_pf/data_link_strict/0194.h5
Finished reading block
Finished operations with block 2 in 106.589503 seconds
Total runtime so far: 319.170823 seconds
Appending the 3th block: /mnt/hp03/donglai/public/cere_pf/data_link_strict/0284.h5
Finished reading block
Finished operations with block 3 in 105.397472 seconds
Total runtime so far: 424.568295 seconds
Saving superblock with 4 blocks
Appending the 4th block: /mnt/hp03/donglai/public/cere_pf/data_link_strict/0374.h5
Finished reading block
Finished operations wi

In [3]:
del superblock # clear memory

In [12]:
# downsize superblock further for visualization
resolution_vis = (96,96,30)
superblock_vis = read3d_h5(save_folder+save_fname+ '_%dblocks.h5'%(n_blocks), 'main', dsmpl=(1,2,2))
print superblock_vis.shape
# zero every 3rd z-slice, so effective z-resolution is 90nm
for z in range(0,superblock_vis.shape[0],3):
    superblock_vis[z,:,:] = 0
writeh5(save_folder+save_fname+'_vis.h5', 'main', superblock_vis, compression="gzip")

(2340, 271, 580)


## Full volume GT skeletonization

In [17]:

# get gt shape and offset
gt48nm_file = data_locs["gt"]["dir"] + data_locs["gt"]["48nm"]
bbox = data_locs["gt"]["48nm-bbox"]
gt_superblock = SegPrep("gt-all-slices", resolution)
block_lims = ((bbox[0],bbox[3]),(bbox[1],bbox[4]),(bbox[2],bbox[5]))
print block_lims
gt_superblock.read(gt48nm_file, "main", block_lims=block_lims)
print gt_superblock.shape
# use relabeling id map of GT generated using single block
id_map = np.load('/home/srujanm/cerebellum/segs/gt0014/relabeling-map.npy')
gt_superblock.relabel(id_map=id_map, print_labels=False)
gt_superblock.write()

((0, 1001), (16, 556), (208, 696))
[1001, 540, 488]
Starting relabeling of 1029 objects
Relabeling time: 0.000296


In [18]:
# skeletonize GT
gt_sb_name = "gt-all-slices"
from cerebellum.skeletonize import gen_skeletons
gen_skeletons(gt_sb_name, overwrite_prev=False)

Starting skeletonization of gt-all-slices
Downsampling to resolution (80, 80, 80) in 8.20758390427 seconds
Topological thinning time for (80, 80, 80): 84.4062309265
Endpoint vector time for (80, 80, 80): 67.6043131351
Edge finding time for (80, 80, 80): 56.4648480415


## PF segmentation superblocks cropped and aligned with GT

Restart kernel here and start afresh to avoid potential errors!

Here, we generate GT superblocks (one time evaluation and save) and pred superblocks cropped and aligned to GT for error analysis

### GT superblock generation

In [ ]:
import json
import time
import math
from cerebellum.data_prep.seg_prep import *

with open('data_locs.json') as f:
	data_locs = json.load(f)
    


In [2]:
import json
import time
import math
from cerebellum.data_prep.seg_prep import *

with open('data_locs.json') as f:
	data_locs = json.load(f)
    
linked_block_file = data_locs["initial-seg"]["dir"]+data_locs["initial-seg"]["8nm-pf-linked"] # first block f
resolution = (30,48,48) # order Z, Y, X
dsmpl = (1,6,6)

# get gt shape and offset
gt8nm_file = data_locs["gt"]["dir"] + data_locs["gt"]["8nm"]
gt_shape = h5py.File(gt48nm_file,'r')['main'].shape
print "Downsampled GT size:", gt_shape
offset = data_locs["initial-seg"]["8nm-offset"] # offset aligns segmentation with GT
bbox_8nm = data_locs["gt"]["8nm-bbox"]

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Downsampled GT size: (1001, 625, 1250)


In [ ]:
# generate pred superblocks cropped to GT extents
cropped_folder = './data/superblocks_haidong/'
cropped_fname = 'pred_crop2gt_superblock'

eval_blocks = 12 # max number of blocks for GT evaluation
eval_superblock_steps = [2, 4, 8, 12] # save after assembling these many blocks
eval_superblock = np.zeros((eval_blocks*gt_shape[0], 
                            gt_shape[1], gt_shape[2]), dtype=np.uint32) # allocate max size
total_runtime = 0
for i in range(eval_blocks):
    block_time = time.time()
    zz = i*data_locs["block-size"]+data_locs["aff-offset"]
    if zz!=data_locs["aff-offset"]: # adjust block index
        linked_block_file = linked_block_file[:-7]+"%04d.h5"%(zz)
    print "Appending the %dth block: "%(i) + linked_block_file
    seg_block = SegPrep("temp", resolution)
    seg_block_lims = ((0,gt_shape[0]),
                      (max(0,bbox[1]+offset[1]),bbox[4]+offset[1]),
                      (max(0,bbox[2]+offset[2]),bbox[5]+offset[2]))
    seg_block.read(linked_block_file, "main", dsmpl=dsmpl, block_lims=seg_block_lims)
    # pad zeros if required
    for pad_axis in [1,2]:
        extra_zeros = gt_shape[pad_axis]-seg_block.shape[pad_axis]
        if extra_zeros!=0: seg_block.padzeros(0,pad_axis)
    print "Finished reading block"
    eval_superblock[i*gt_shape[0]:(i+1)*gt_shape[0],:,:] = seg_block.data
    block_runtime = time.time()-block_time
    print "Finished operations with block %d in %f seconds"%(i, block_runtime)
    total_runtime += block_runtime
    print "Total runtime so far: %f seconds"%(total_runtime)
    if i+1 in eval_superblock_steps:
        print "Saving superblock with %d blocks"%(i)
        writeh5(cropped_folder+cropped_fname+ '_%dblocks.h5'%(i))